# spatial visualization

The following code is used for preprocessing assstance, reverse geocoding for location confirmation and zipcode generation, and a preliminary exploration into the relative distibution of fatal crashes to one another. To access files used in this colab, download the contents of the zipcodes folder and the WTSC Fatal Crash Files. You will need to attach the files directly to the colab file via uploading to session storage. 

### Install necessary libraries for spacial visualization

In [ ]:
!pip install --upgrade geopandas
!pip install geopy
!pip install -U mapclassify
!pip install libpysal
import folium, matplotlib, mapclassify
import libpysal
import geopy as geo
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from numpy import NaN
import matplotlib.pyplot as plt

Run this cell to connect to your google drive, the extension in drive.mount will be used to store modified files in your drive. Alternatively, without google drive you can view output files on the far left folder icon. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Loading the Dataset

In [6]:
zcta_shp = gpd.read_file('cb_2018_us_zcta510_500k.shp') # Load the shapefile
df = pd.read_csv('Data_Washington Fatal Crash Survey.csv', low_memory=False) # mixed column attribute type, include to prevent pandas from guessing data types.

#### Preprocessing Fatal Crash Files

In [7]:
# Keep only necessary Columns     **** Need to go and remove records with invalid coords outright
#accidents=df[['x','y']].copy()
accidents=df[['x','y','numfatal']].copy()

#Removes erranous coordinates
accidents = accidents[abs(accidents.x) <= 180] 
accidents = accidents[abs(accidents.y) <= 90]

#Convert to point geometry
accidents['geometry'] = accidents.apply(lambda x: Point((float(x.x), float(x.y))), axis=1)

#drop origional coords keep index**
accidents = accidents.drop(['x','y'], axis=1)

#Convert to geo df
accidents = gpd.GeoDataFrame(accidents, geometry='geometry')

### Preprocessing the Shapefile

In [ ]:
pd.set_option.mode_use_inf_as_na = True #enables alt empty and other na variations to be accepted as 'NaN'

In [8]:
#Convert to geo dataframe
zcta_shp = gpd.GeoDataFrame(zcta_shp, geometry='geometry')

#### ZTCA

In [10]:

# check same for accidnets 
zcta_shp['geometry']

# Keep only the necessary columns
zcta_shp = zcta_shp[['ZCTA5CE10', 'geometry']]
# Rename the columns
zcta_shp.columns = ['zip_code', 'geometry']
# Drop any invalid geometries
zcta_shp = zcta_shp[zcta_shp.geometry.is_valid]
# Drop any duplicates
zcta_shp = zcta_shp.drop_duplicates(subset='zip_code', keep='first')
#Filter out zipcodes outside of Washington and local areas - Value confirmed externally
zcta_shp = zcta_shp[zcta_shp['zip_code'] > '90000']

In [ ]:
# Check for inconsistencies
if not zcta_shp.is_valid.all():
    zcta_shp = zcta_shp.buffer(0)  # Fix invalid polygons
    print("Inconsistent polygons found and fixed.")
else:
    print("No inconsistent polygons found.")

###### Test compadibility, clean, and process shape file and crash files

In [ ]:
accidents['geometry'].head()
invalid_geoms = accidents.geometry[~accidents.geometry.is_valid]
empty_geoms = accidents.geometry[accidents.geometry.is_empty]

print("Invalid geometries:", len(invalid_geoms))
print("Empty geometries:", len(empty_geoms))

accidents['geometry'].isna().count()

## Vizualizations --> spatial

#### Interactive Visualization -- Usable for intial exploration of Relative Position

In [ ]:
the_end = gpd.GeoDataFrame.explore(accidents, cmap="Set2", )
the_end

#### Static Plot - Number of fatalites by relative location

In [ ]:
acc = accidents.plot(column='numfatal',cmap="Set2", figsize=(30,10), legend=True)

In [16]:
# store plot output in session storage
acc.figure.savefig('output5.png') #click on folder, left click on the name of the file and click download for your physcial static plot

### Reverse Geocode useing ARCGIS sub server

In [ ]:
#available services and subservices for reverse geocoding through geodataframe. Use a service you have access too for effecient processing, otherwise: consider using googleV3 or nominatim 
geo.geocoders.SERVICE_TO_GEOCODER

In [ ]:
# reverse geocoding, returns dataframe with full address in new column
from shapely.geometry import Point
zip = gpd.tools.reverse_geocode(accidents['geometry'], provider='arcgis', timeout=1000)


In [ ]:
###Make Copy of zips and move forward
zips = zip.copy()
zip[zip['address'].isna()==True]

,geometry,address


#### Store zipcodes in csv file for further reference and integration into crash files. 

In [ ]:
accidents[['geometry1','address']] = zip[['geometry','address']]

In [ ]:
zip.to_csv('zip_coord.csv')
!cp zip_coord.csv "drive/My Drive/"

#### Check Reverse Geocoding was complete. an filtered dataframe with no values is the expected output. If you have any other output under the column headers of the dataframe, 

## export files

In [ ]:
gpd.sjoin(accidents, zip, how='right')

In [ ]:
# Save the cleaned shapefile as a GeoJSON file
zcta_shp.to_file('zcta_cleaned.geojson', driver='GeoJSON')
accidents.to_csv('accidents.csv')